In [1]:
from langchain import LLMChain, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0)

In [4]:
template = """
Answer the question based on the context given below. If the question can not be answered with the provided context then answer 
with I dont know. 

Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.

question: {question}
answer: 
"""

prompt = PromptTemplate(template= template, input_variables=['question'])

In [5]:
chain = LLMChain(prompt= prompt, llm=llm)

In [6]:
input_data1 = {"question": "What is quantum computing?"}
input_data2 = {"question": "What is World highest moutain?"}

response1 = chain.run(input_data1)
response2 = chain.run(input_data2)

In [7]:
print(response1)

Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.


In [8]:
print(response2)

I don't know. The provided context does not contain any information about the world's highest mountain.


# Creating a Scarcastic Ai Assistant

In [10]:
from langchain import FewShotPromptTemplate

examples = [
    {
        "query": "How do I become a better programmer?",
        'answer': "Try talking to a rubber duck; it works wonders."
    },
    {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    template= example_template,
    input_variables = ['query', 'answer']
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples:"""

suffix = """User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator="\n\n"
)

In [11]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

In [12]:
input_data = {"query": "How can I learn quantum computing?"}
response = chain.run(input_data)

In [13]:
print(response)

By reading tea leaves and staring at a crystal ball.
